In [ ]:
import pandahouse as ph
from airflow import DAG
from airflow.operators.python_operator import PythonOperator # Так как мы пишет такси в питоне
from datetime import datetime,  timedelta
from airflow.decorators import dag, task
from airflow.operators.python import get_current_context 
import telegram
import matplotlib.pyplot as plt
import seaborn as sns
import io
import pandas as pd
from datetime import datetime,  timedelta

# подключение к БД
connection = {'host': '*****',
              'database':'*****',
              'user':'*****', 
              'password':'*****'
              }

schedule_interval = '0 7 * * *' # cron - выражение. Каждый день в 7 утра

# дефолтные аргументы для DAG
default_args = {
    'owner': 'r_muksinov', # владелец
    'depends_on_past': False, # не зависит от  успешности прошлого запуска 
    'retries': 2, # количество рестартов
    'retry_delay': timedelta(minutes=5), # пауза между рестартами
    'start_date': datetime(2023, 4, 9) # начало выполнения
}
 
chat_id = *****
my_token = '*****' 
bot = telegram.Bot(token=my_token) # получаем доступ


@dag(default_args=default_args, schedule_interval=schedule_interval, catchup=False, tags=['r.muksinov'])
def muksinov_metrics_bot():
    @task()
    def get_day_metrics():
        # основные метрики за прошедший день
        query1 = '''
                    select toString(toDate(time)) as event_date,
                           count(distinct user_id) as DAU, 
                           countIf(action='like') AS likes, 
                           countIf(action='view') AS views, 
                           likes/views as CTR
                    from simulator_20230320.feed_actions 
                    where toDate(time) = today() - 1 
                    group by toString(toDate(time))
                '''
        df_yesterday = ph.read_clickhouse(query1, connection=connection)

        date = list(df_yesterday['event_date'].values)[0]
        DAU = df_yesterday['DAU'].values[0]
        like = df_yesterday['likes'].values[0]
        views = df_yesterday['views'].values[0]
        CTR = df_yesterday['CTR'].values[0].round(3)
        msg = f'Отчет за {date}\nDAU: {DAU}\nЛайки: {like}\nПросмотры: {views}\nCTR: {CTR}\n'
    
        return msg
    
    @task()
    def get_week_metrics():
        # основные метрики за последнюю неделю
        query2 = '''
                    select toString(toDate(time)) as event_date,
                            count(distinct user_id) as DAU, 
                            countIf(action='like') AS likes, 
                            countIf(action='view') AS views, 
                            likes/views as CTR
                    from simulator_20230320.feed_actions 
                    where toDate(time) between yesterday() - 6 and yesterday()
                    group by toString(toDate(time))
                '''
        
        df_week = ph.read_clickhouse(query2, connection=connection)
        
        return df_week

     
    sns.set_style('darkgrid', {'axes.facecolor': '.95'})
    fig, axes = plt.subplots(2, 2, figsize=(14, 14))

    @task()
    def visualise(data, message):
        # DAU
        sns.lineplot(data=data, x='event_date', y='DAU', ax=axes[0, 0])
        axes[0, 0].set_title('DAU', fontsize=18)
        axes[0, 0].set_ylabel('уникальные пользователи', fontsize=14)
        axes[0, 0].set_xlabel('')
        axes[0, 0].tick_params(axis='x', labelsize=14, rotation=90)
        
        # Лайки
        sns.lineplot(data=data, x='event_date', y='likes', ax=axes[0, 1])
        axes[0, 1].set_title('Лайки', fontsize=18)
        axes[0, 1].set_ylabel('кол-во', fontsize=14)
        axes[0, 1].set_xlabel('')
        axes[0, 1].tick_params(axis='x', labelsize=14, rotation=90)
        
        # Просмотры
        sns.lineplot(data=data, x='event_date', y='views', ax=axes[1, 0])
        axes[1, 0].set_title('Просмотры', fontsize=18)
        axes[1, 0].set_ylabel('кол-во', fontsize=14)
        axes[1, 0].set_xlabel('')
        axes[1, 0].tick_params(axis='x', labelsize=14, rotation=90)

        # CTR
        sns.lineplot(data=data, x='event_date', y='CTR', ax=axes[1, 1])
        axes[1, 1].set_title('CTR', fontsize=18)
        axes[1, 1].set_ylabel('', fontsize=14)
        axes[1, 1].set_xlabel('')
        axes[1, 1].tick_params(axis='x', labelsize=14, rotation=90)

        plt.tight_layout()
        plot_object = io.BytesIO()
        plt.savefig(plot_object, dpi=200)
        plot_object.seek(0)
        plot_object.name = 'key_metrics.png'
        plt.close()
        bot.sendPhoto(chat_id=chat_id, photo=plot_object, caption=message)

    
    message = get_day_metrics()
    df_week = get_week_metrics()
    visualise(df_week, message)

    
muksinov_metrics_bot = muksinov_metrics_bot()